In [138]:
import os
import pandas as pd

# Turn off warnings
import warnings
warnings.filterwarnings('ignore')
# File list 
file_directories = []

# List and save every file name in the input directory
for file in os.listdir('../Data'):
    print('../Data/', file, sep='/')
    file_directories.append(file)


../Data//cleaned_merged_data.xlsx
../Data//us-shein-appliances-3987.csv
../Data//us-shein-automotive-4110.csv
../Data//us-shein-baby_and_maternity-4433.csv
../Data//us-shein-bags_and_luggage-4299.csv
../Data//us-shein-beauty_and_health-4267.csv
../Data//us-shein-curve-2849.csv
../Data//us-shein-electronics-4395.csv
../Data//us-shein-home_and_kitchen-3719.csv
../Data//us-shein-home_textile-3883.csv
../Data//us-shein-jewelry_and_accessories-3548.csv
../Data//us-shein-kids-4314.csv
../Data//us-shein-mens_clothes-1891.csv
../Data//us-shein-office_and_school_supplies-4233.csv
../Data//us-shein-pet_supplies-4083.csv
../Data//us-shein-shoes-4381.csv
../Data//us-shein-sports_and_outdoors-3853.csv
../Data//us-shein-swimwear-3761.csv
../Data//us-shein-tools_and_home_improvement-3903.csv
../Data//us-shein-toys_and_games-3577.csv
../Data//us-shein-underwear_and_sleepwear-4019.csv
../Data//us-shein-womens_clothing-4620.csv


In [139]:
# Path where files are stored
data_folder = "../Data/"
prefix = "us-shein-"

# Create a list of DataFrame names
dfs = {}
df_names = []

# Iterate through each file and category name
for file in os.listdir(data_folder):
    if file.startswith(prefix) and file.endswith(".csv"):
        category = file.replace(prefix, "").replace(".csv", "")
        file_path = os.path.join(data_folder, file)

        dfs[category] = pd.read_csv(file_path)
        df_names.append(category)

df_names

['appliances-3987',
 'automotive-4110',
 'baby_and_maternity-4433',
 'bags_and_luggage-4299',
 'beauty_and_health-4267',
 'curve-2849',
 'electronics-4395',
 'home_and_kitchen-3719',
 'home_textile-3883',
 'jewelry_and_accessories-3548',
 'kids-4314',
 'mens_clothes-1891',
 'office_and_school_supplies-4233',
 'pet_supplies-4083',
 'shoes-4381',
 'sports_and_outdoors-3853',
 'swimwear-3761',
 'tools_and_home_improvement-3903',
 'toys_and_games-3577',
 'underwear_and_sleepwear-4019',
 'womens_clothing-4620']

In [140]:
# Initialize counter
count = 0

# Clean each DataFrame and print progress
for count, (category, df) in enumerate(dfs.items(), start=1):
    dfs[category] = clean_data(df)
    print(count, f"- No Warnings in {category}")

1 - No Warnings in appliances-3987
2 - No Warnings in automotive-4110
3 - No Warnings in baby_and_maternity-4433
4 - No Warnings in bags_and_luggage-4299
5 - No Warnings in beauty_and_health-4267
6 - No Warnings in curve-2849
7 - No Warnings in electronics-4395
8 - No Warnings in home_and_kitchen-3719
9 - No Warnings in home_textile-3883
10 - No Warnings in jewelry_and_accessories-3548
11 - No Warnings in kids-4314
12 - No Warnings in mens_clothes-1891
13 - No Warnings in office_and_school_supplies-4233
14 - No Warnings in pet_supplies-4083
15 - No Warnings in shoes-4381
16 - No Warnings in sports_and_outdoors-3853
17 - No Warnings in swimwear-3761
18 - No Warnings in tools_and_home_improvement-3903
19 - No Warnings in toys_and_games-3577
20 - No Warnings in underwear_and_sleepwear-4019
21 - No Warnings in womens_clothing-4620


In [141]:
def clean_data(df):
    '''
    This function takes in a category table of products and cleans it.
    
    Args: 
        df: DataFrame containing product data for a category.
    
    Returns: 
        df: A cleaned DataFrame.
    '''
    col_names = {}
    
    # Handle missing goods title column
    if ('goods-title-link' in df.columns) and ('goods-title-link--jump' in df.columns):
        # Merge the --jump goods titles and rename
        df.loc[:, 'goods-title-link'] = df['goods-title-link'].fillna(df['goods-title-link--jump'])

        # Drop unnecessary columns
        df = df.drop(columns=['goods-title-link--jump', 
                          'goods-title-link--jump href'])

    # Fill Discount NaNs and remove '%'
    df.loc[:, 'discount'] = df['discount'].fillna(0)
       # Handle missing selling proposition column
    if 'selling_proposition' in df.columns:
        df.loc[:, 'selling_proposition'] = df['selling_proposition'].str.replace('+ sold recently', '')
        col_names['selling_proposition'] =  'qty_sold'
        
    # Handle missing rank subcategory column
    if 'rank-sub' in df.columns:    
        df.loc[:, 'rank-sub'] = df['rank-sub'].str.replace('in', '')
        col_names['rank-sub'] =  'rank_category'
    
    # Handle missing color count column
    if 'color-count' in df.columns:
        col_names['color-count'] = 'color_count'
    
    # Handle black friday sale column
    if 'blackfridaybelts-bg src' in df.columns:
        df = df.drop(columns='blackfridaybelts-bg src')
        df['blackfridaybelts-content'] = df['blackfridaybelts-content'].str.replace('Save $', '')
        col_names['blackfridaybelts-content'] =  'black_friday_off_usd'
    
    if 'product-locatelabels-img src' in df.columns:
        col_names['product-locatelabels-img src'] = 'img_source_url'
    # Replace unnecessary text in columns
    df.loc[:, 'discount'] = df['discount'].str.replace('%', '')
    df.loc[:, 'price'] = df['price'].str.replace('$', '')


    # Rename columns
    col_names.update({'rank-title': 'rank',
                 'price': 'price_usd', 
                 'discount': 'pct_discount', 
                 'goods-title-link': 'product_title'})
    df = df.rename(columns=col_names)

    # Handle missing quantity sold column
    if 'qty_sold' in df.columns:
        # Change the 'k' to thousands 
        for col_num, col in enumerate(df.qty_sold):
            if 'k' in str(col):
                num = col.replace('k', '')
                df.qty_sold.iloc[col_num] = int(float(num)*1000)
    
    # pct_discount to absolute value
    df['pct_discount'] = [abs(float(disc)) for disc in df['pct_discount']]
    
    return df

In [142]:
df_names

['appliances-3987',
 'automotive-4110',
 'baby_and_maternity-4433',
 'bags_and_luggage-4299',
 'beauty_and_health-4267',
 'curve-2849',
 'electronics-4395',
 'home_and_kitchen-3719',
 'home_textile-3883',
 'jewelry_and_accessories-3548',
 'kids-4314',
 'mens_clothes-1891',
 'office_and_school_supplies-4233',
 'pet_supplies-4083',
 'shoes-4381',
 'sports_and_outdoors-3853',
 'swimwear-3761',
 'tools_and_home_improvement-3903',
 'toys_and_games-3577',
 'underwear_and_sleepwear-4019',
 'womens_clothing-4620']

In [143]:
all_unique_columns = set()

for df in dfs.values():
    all_unique_columns.update(df.columns)

all_unique_columns

{'black_friday_off_usd',
 'color_count',
 'img_source_url',
 'pct_discount',
 'price_usd',
 'product_title',
 'qty_sold',
 'rank',
 'rank_category'}

In [144]:
all_dfs = []

for category, df in dfs.items():
    all_dfs.append(add_category_name(df.copy(), category))

merged_df = pd.concat(all_dfs, join='outer', ignore_index=True)

print("Merged DataFrame shape:", merged_df.shape)

Merged DataFrame shape: (82105, 10)


In [145]:
# Clean the rank column
rank_num = merged_df[['rank']].assign(
    first_part = lambda x: x['rank'].str.split(' ').str.get(0)
)
merged_df['rank'] = rank_num['first_part']

In [146]:
merged_df.head()

,rank,rank_category,price_usd,pct_discount,qty_sold,product_title,category_name,color_count,black_friday_off_usd,img_source_url
0,#1,Give Gifts,2.03,22.0,NaN,1pc Rechargeable Deep Tissue Muscle Handheld M...,appliances-3987,NaN,NaN,NaN
1,#4,Top rated Portable Fans,6.48,20.0,NaN,1pc Portable Hanging Neck Fan,appliances-3987,NaN,NaN,NaN
2,NaN,NaN,1.80,NaN,400,1pc Pink Colored Curved Eyelash Curler False E...,appliances-3987,NaN,NaN,NaN
3,NaN,NaN,0.88,72.0,5600,1 Mini Portable Handheld Fan With 2 Aa Batteri...,appliances-3987,NaN,NaN,NaN
4,#6,Oral Irrigators,12.06,40.0,NaN,"Wit Water Flosser,Portable Oral Irrigator With...",appliances-3987,NaN,NaN,NaN


In [147]:
merged_df


,rank,rank_category,price_usd,pct_discount,qty_sold,product_title,category_name,color_count,black_friday_off_usd,img_source_url
0,#1,Give Gifts,2.03,22.0,NaN,1pc Rechargeable Deep Tissue Muscle Handheld M...,appliances-3987,NaN,NaN,NaN
1,#4,Top rated Portable Fans,6.48,20.0,NaN,1pc Portable Hanging Neck Fan,appliances-3987,NaN,NaN,NaN
2,NaN,NaN,1.80,NaN,400,1pc Pink Colored Curved Eyelash Curler False E...,appliances-3987,NaN,NaN,NaN
3,NaN,NaN,0.88,72.0,5600,1 Mini Portable Handheld Fan With 2 Aa Batteri...,appliances-3987,NaN,NaN,NaN
4,#6,Oral Irrigators,12.06,40.0,NaN,"Wit Water Flosser,Portable Oral Irrigator With...",appliances-3987,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
82100,#2,In Modest World Apparel,35.69,NaN,NaN,Colorful Flower Embroidered Linen Muslim Women...,womens_clothing-4620,NaN,NaN,NaN
82101,NaN,NaN,42.49,NaN,10,Elegant And Stylish Patchwork Lace Cheongsam D...,womens_clothing-4620,NaN,NaN,NaN
82102,NaN,NaN,49.29,NaN,200,Split Thigh Satin Tube Prom Dress,womens_clothing-4620,5.0,NaN,NaN
82103,#10,In Sexy Women Bottoms,9.78,5.0,NaN,DAZY Women's Solid Color Low Waist Ultra Short...,womens_clothing-4620,6.0,NaN,NaN


In [ ]:
output_path = "../Data/cleaned_merged_data.xlsx"
merged_df.to_excel(output_path, index=False)
print(f"Cleaned dataset saved to {output_path}")

In [160]:
# Path to your cleaned Excel
file_path = "../Data/cleaned_merged_data.xlsx"

# Read Excel
df_check = pd.read_excel(file_path)

# Quick check
print(df_check.head())
print(df_check.shape)


  rank              rank_category price_usd  pct_discount  qty_sold  \
0   #1                 Give Gifts      2.03          22.0       NaN   
1   #4   Top rated  Portable Fans      6.48          20.0       NaN   
2  NaN                        NaN      1.80           NaN     400.0   
3  NaN                        NaN      0.88          72.0    5600.0   
4   #6            Oral Irrigators     12.06          40.0       NaN   

                                       product_title    category_name  \
0  1pc Rechargeable Deep Tissue Muscle Handheld M...  appliances-3987   
1                      1pc Portable Hanging Neck Fan  appliances-3987   
2  1pc Pink Colored Curved Eyelash Curler False E...  appliances-3987   
3  1 Mini Portable Handheld Fan With 2 Aa Batteri...  appliances-3987   
4  Wit Water Flosser,Portable Oral Irrigator With...  appliances-3987   

   color_count  black_friday_off_usd img_source_url  
0          NaN                   NaN            NaN  
1          NaN            

In [163]:
df_check["color_count"][0].type

AttributeError: 'numpy.float64' object has no attribute 'type'

In [164]:
df_check.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82105 entries, 0 to 82104
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   rank                  14605 non-null  object 
 1   rank_category         14605 non-null  object 
 2   price_usd             82103 non-null  object 
 3   pct_discount          54977 non-null  float64
 4   qty_sold              54364 non-null  float64
 5   product_title         82091 non-null  object 
 6   category_name         82105 non-null  object 
 7   color_count           19743 non-null  float64
 8   black_friday_off_usd  3791 non-null   float64
 9   img_source_url        3622 non-null   object 
dtypes: float64(4), object(6)
memory usage: 6.3+ MB
